In [1]:
!pip install ffmpeg-python
!pip install transformers torch
!pip install wheel
!pip install tts

In [7]:

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [26]:
audio, sr = get_audio()

In [27]:
import scipy
scipy.io.wavfile.write('recording.wav', sr, audio)

In [5]:
!pip install SpeechRecognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 32.4 MB/s eta 0:00:00


In [2]:
import speech_recognition as sr
r = sr.Recognizer()

hellow=sr.AudioFile('recording.wav')
with hellow as source:
    audio = r.record(source)
try:
    s = r.recognize_google(audio)
    print("Text: "+s)
except Exception as e:
    print("Exception: "+str(e))

Text: I need your help


In [6]:
import random
import bisect
responses = {
    "hello": ["Hi there!", "Hello!", "Hey!"],
    "hyy": ["hey! whats up? hows your day going?"],
    "good": ["that's awesome to hear"],
    "bad":["I'm sorry to hear that you're not feeling so good. If you want to talk about it or need a distraction, I'm here for you. Just let me know how I can help!"],
    "sure":["I'm here for you, buddy. Feel free to share what's on your mind. Sometimes talking about it can help lighten the load."],
    "thanks for being there":["Of course, anytime! I'm here to listen and support you. "],
    "you are so sweet": ["Aww, thank you! You're making me blush. 😊 Just trying to be a good friend to you!"],
    "how are you": ["I'm a chatbot, so I'm always good!", "Doing great, thanks!", "I'm here to help you!"],
    "i an fine": ["that's great to hear!is there anything exciting?"],
    "i am hungry":["i have some sites you can order from https://www.swiggy.com here","just swiggy it! https://www.swiggy.com/"],
    "i want to explore about something ": ["you can go to the browsers like https://www.wikipedia.org and https://www.google.com....click any of the following links"],
    "can you help me?":["Absolutely, I'm here to help you out! What do you need assistance with? Just tell me, and I'll do my best to support you."],
    "what can you do for me?": ["I can help answer your questions, give recommendations, chat with you, and share fun facts or tips. Just let me know what you're looking for, and I'll do my best to assist you!"],
    "i want to see the news": ["sure,here are some of popular news channels such as https://www.indiatoday.in, https://www.aajtak.in and https://www.ndtv.com"],
    "what task you are capable to do": ["I can do so many things","i can answer your question","ask me anything"],
    "your name": ["I'm a chatbot created by OpenAI.", "I am a simple Python chatbot.", "Just call me Chatbot!"],
    "open instagram": ["just click on the link https://www.instagram.com"],
    "i am lookin for sports": ["What kind of sports are you interested in? Football, cricket, basketball, or something else? Let me know, and I can share some cool info or updates with you!"],
    "open twiter": ["just click on the link https://twitter.com"],
    "i am lost...please show me the direction": ["here is the google maps...help you to find a way... https://maps.google.com"],
    "bye": ["Goodbye!", "See you later!", "Take care!"],
    "i need your help": ["Of course, I'm here to help! What do you need assistance with?"],
    "see you again!": ["See you later! If you ever want to chat or share something, I'll be here.Take care!"],
    "default": ["I'm not sure how to respond to that.", "Can you please rephrase?", "I don't understand that."]
}
def get_response(user_input):
    user_input = user_input.lower()
    for key in responses:
        if key in user_input:
            return random.choice(responses[key])
    return random.choice(responses["default"])


sorted_keys = sorted(responses.keys())

# Binary search function
def binary_search(keys, target):
    index = bisect.bisect_left(keys, target)
    if index < len(keys) and keys[index] == target:
        #return random.choice(keys[index])
        return random.choice(responses[keys[index]])
    return -1

# Searching for 'banana'
output=get_response(s)
print(output)

Of course, I'm here to help! What do you need assistance with?


In [8]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False)

# generate speech by cloning a voice using default settings
tts.tts_to_file(text=output,
                file_path="output.wav",
                speaker_wav="preamble.wav",
                language="en")
display(Audio("output.wav", autoplay=True))


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts
 > Text splitted to sentences.
["Of course, I'm here to help!", 'What do you need assistance with?']
 > Processing time: 39.65047335624695
 > Real-time factor: 7.795051154647336
